<a href="https://colab.research.google.com/github/alahirajeffrey/Introduction-to-Covnets/blob/master/Transfer_learning_using_ResNet50_and_cifar10_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
import tensorflow as tf

In [2]:
#import and load dataset
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
def process_data(x, y):
  """
  process input data using resnet50
  """

  x_train = keras.applications.resnet50.preprocess_input(x)
  y_train = keras.utils.to_categorical(y)

  return x_train, y_train

In [6]:
x_train, y_train = process_data(x_train, y_train)
x_test, y_test = process_data(x_test, y_test)

In [7]:
##check shape of dataset
print('shape of x_train : ', x_train.shape)
print('shape of y_train : ', y_train.shape)
print('shape of x_test : ', x_test.shape)
print('shape of y_test : ', y_test.shape)

shape of x_train :  (50000, 32, 32, 3)
shape of y_train :  (50000, 10)
shape of x_test :  (10000, 32, 32, 3)
shape of y_test :  (10000, 10)


In [8]:
#set input shape of dataset
input_shape = keras.Input(shape = (32,32,3))

#load resnet model
res_model = keras.applications.ResNet50(include_top =False, weights = 'imagenet',   input_tensor = input_shape)

In [9]:
#check number of layers in resnet50 model
len(res_model.layers)

175

In [11]:
res_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [16]:
#freeze first 143 layers
for layer in res_model.layers[:143]:
  layer.trainable = False

In [29]:
#build second model and add to first resnet model
model = keras.models.Sequential()
model.add(res_model)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(10, activation='softmax'))

#compile model
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
#create model callback
from keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(patience=2)

In [ ]:
#fit model
model.fit(x_train, y_train, batch_size=32, epochs=8, verbose=1, validation_data=(x_test, y_test), callbacks=[early_stopping_monitor])

Epoch 1/10
1563/1563 [==============================] - 2028s 1s/step - loss: 1.5676 - accuracy: 0.4686 - val_loss: 1.0519 - val_accuracy: 0.6567
Epoch 2/10
1563/1563 [==============================] - 2033s 1s/step - loss: 1.2075 - accuracy: 0.6156 - val_loss: 0.9800 - val_accuracy: 0.6870
Epoch 3/10
 634/1563 [===========>..................] - ETA: 19:39 - loss: 1.0937 - accuracy: 0.6532